In [2]:
import math
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import csv
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import polynomial_kernel
from numpy.linalg import inv, norm
from mpl_toolkits.mplot3d import axes3d
from numpy import mean
%matplotlib inline

### Import data from csv
Store the data as a numpy array. \
Shape of the full dataset: 
9298 observations (rows), 257 cols, where column 1 corresponds to the true label of the digit, while the cols 2:257 store greyscale values.


In [3]:
# Import to df
data_df = pd.read_csv("zipcombo2.csv")

# Convert to array to facilitate vectorization later
data = data_df.to_numpy()
X = data[:, 1:]
y = data[:, 0]

## Define useful functions

In [5]:
def efficient_poly_matrix(X_i, X_j, d):
    
    K = polynomial_kernel(X_i, Y=X_j, degree=d)
    
    return K 

In [6]:
def gaussian_matrix(X, X_, d):
    
    K = euclidean_distances(X,X_)
    K_mat = np.exp(-d*K**2)
    
    return K_mat

In [7]:
def sign(pred):
    
    if pred <= 0:
        return -1
    else:
        return 1

## Pre-compute indexing conventions for the 45 pairwise classifiers

**Classification convention**

We use a convention whereby the digit in the first column is treated as the positive class and the digit in the second column as the negative class. 

For instance, for a [3,8] classifier 3 is thresholded as +1 and 8 as -1.



**classifiers: The pairs considered in each of the 45 classifiers**

In [8]:
combos = []
for i in range(10):
    for j in range(10):
        if i != j:
            combos.append([i,j])
for i in combos:
    i.sort()
classifiers = [list(y) for y in set([tuple(x) for x in combos])]
classifiers.sort()

#### digit_indexes: which alphas to index for each digit

In [9]:
# Compute the indexes for each of the digits
# Each of the nested lists corresponds to a specific digit.
# eg. digit_indexes[8] contains the indexes of all the  
# classifiers which classify for 8

digit_indexes = []
for digit in range(10):
    digit_ind = []
    for j in classifiers:
        if digit in j:
            digit_ind.append(classifiers.index(j))
    digit_indexes.append(digit_ind)

#### digit_classifiers: the 2 digits each classifier considers

In [10]:
classifiers = np.array(classifiers)
digit_classifiers = []
for i in range(10):
    d_classifs = classifiers[digit_indexes[i]].tolist()
    digit_classifiers.append(d_classifs)

#### digit_signs: whether the given digit is treated as the positive or negative class

In [11]:
digit_signs = []
for i in range(10):
    d_sign = []
    for pair in digit_classifiers[i]:
        
        if pair[0] == i :
            d_sign.append(+1)
        else:
            d_sign.append(-1)
    digit_signs.append(d_sign)

### K-class 1-vs-1 perceptron. 

We find that the algorithm converges after approximately 8-10 epochs, where by 'convergence' we refer to convergence on the test set.

In [12]:
class KernelPerceptron(object):
    
    def __init__(self, X_train, y_train, X_test, y_test, kernel_type='p', d=5, epochs = 8):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.kernel_type = kernel_type
        self.degree = d
        self.epochs = epochs
        
        # Initalise number of classes, number of classifiers and batch size
        self.C = 10
        self.c = 45
        self.batch_size = self.y_train.shape[0]  
        self.test_size = self.y_test.shape[0]

        # Compute the kernel matrix
        if kernel_type == 'p':
            self.K = efficient_poly_matrix(self.X_train, self.X_train, self.degree)
            self.K_test = efficient_poly_matrix(self.X_train, self.X_test, self.degree)
            
        if kernel_type == 'g':
            self.K = gaussian_matrix(self.X_train, self.X_train, self.degree)
            self.K_test = gaussian_matrix(self.X_train, self.X_test, self.degree)
            
        # Initialise alphas
        self.alphas = np.zeros(shape = (self.batch_size, self.c), dtype = np.float64)
   
        
    def predicition(self, i, t='train'):
       
        # Compute the confidences for each digit for the given observation 
        if t == 'train':
            y_hat = self.alphas.T @ self.K[:,i] 
        
        elif t == 'test':
            y_hat = self.alphas.T @ self.K_test[:,i] 
    
        return y_hat
    
    
    def train_single_pass(self):

        errors = 0

        # For each training example
        for i in range(self.batch_size):

            # True y value
            y = self.y_train[i]
            
            # Get the indexes of classifiers
            digit_ind = digit_indexes[int(y)]
            # Digits which the classifiers look at
            digit_classif = digit_classifiers[int(y)]
            # Whether the classifers treats the digit as the positive or the negative class
            digit_sign =digit_signs[int(y)]

            # Predicted y values for each of the 45 classifiers
            y_predictions = self.predicition(i, t='train')
            
            # Convert predictions to integers
            y_int = np.zeros(shape=(1,45))
            for j in range(len(y_predictions)):
                
                if y_predictions[j]>=0:
                    # Predict the positive class
                    y_int[:,j] = classifiers[j][0]
                else:
                    # Predict the negative class
                    y_int[:,j] = classifiers[j][1]
                    
            
            # Update the 9 relevant classifiers
            y_preds = y_predictions[digit_ind]            
            
            for k in range(len(y_preds)):
                
                # Positive class
                if digit_sign[k] == 1:
                    # If incorrect:
                    if y_preds[k]<=0:
                        # Update alphas
                        self.alphas[i, digit_ind[k]] +=1
                     
                # Negative class    
                else:
                    # If incorrect:
                    if y_preds[k]>=0:
                        # Update alphas
                        self.alphas[i, digit_ind[k]] -=1            
           
            # Final prediction    
            counts = np.bincount(y_int[0,:].astype(int))
            y_final = np.argmax(counts)    
            if y_final != y:
                errors +=1
                
        # Divide by batch size, to make the error invariant to number of observations
        return (errors/self.batch_size)*100  
                
               
    def test(self):
            
        errors = 0

        # For each training example
        for i in range(self.test_size):

            # True y value
            y = self.y_test[i]

            # Predicted y values for each of the 45 classifiers
            y_predictions = self.predicition(i, t='test')
            y_int = np.zeros(shape=(1,45))
            
            for j in range(len(y_predictions)):
                
                if y_predictions[j]>=0:
                    # Predict the positive class
                    y_int[:,j] = classifiers[j][0]
                else:
                    # Predict the negative class
                    y_int[:,j] = classifiers[j][1]

            # Final prediction    
            counts = np.bincount(y_int[0,:].astype(int))
            y_final = np.argmax(counts)    
            
            if y_final != y:
                errors +=1
                      
        # Divide by batch size, to make the error invariant to number of observations
        return (errors/self.test_size)*100  
                
    
    def train(self):
        
        for i in range(self.epochs):
        
            error = self.train_single_pass()
        
        return error

### Time Complexity

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [18]:
times = []
for i in range(20):
    startTime = time.perf_counter()
    
    clf = KernelPerceptron(X_train, y_train, X_test, y_test, kernel_type='p', d=7, epochs=10)
    train_error = clf.train()
    test_error = clf.test()

    t = time.perf_counter() - startTime
    print("Run: " + str(i+1) + " t: " + str(t))
    times.append(t)

Run: 1t: 38.78853180000442
Run: 2t: 46.427496799995424
Run: 3t: 58.51736730002449
Run: 4t: 50.21826880000299
Run: 5t: 62.27371400000993
Run: 6t: 57.97173299998394
Run: 7t: 61.57353999998304
Run: 8t: 66.8907240999979
Run: 9t: 71.13364630000433
Run: 10t: 54.26405440000235
Run: 11t: 37.490868599998066
Run: 12t: 36.099010999983875
Run: 13t: 45.849452599999495
Run: 14t: 35.1863735000079
Run: 15t: 34.20488479998312
Run: 16t: 29.49258029999328
Run: 17t: 37.047727700002724
Run: 18t: 47.630399599991506
Run: 19t: 42.097494900022866
Run: 20t: 49.90988270001253


In [19]:
print("Mean computational time: " + str(np.mean(times)) +"+/-" + str(np.std(times)) + " seconds")

Mean computational time: 48.15338761000021+/-11.690515412047638 seconds


### 1.1 Basic Results

In [50]:
## 20 runs for d = {1,..,8}

train_errors = []
train_sd = []
test_errors = []
test_sd = []

for poly_order in range(1,8):
    
    run_train_errors = []
    run_test_errors = []
    
    for run in range(20):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
        
        clf = KernelPerceptron(X_train, y_train, X_test, y_test, kernel_type='p', d=poly_order, epochs=8)
        train_error = clf.train()
        test_error = clf.test()
        run_train_errors.append(train_error)
        run_test_errors.append(test_error)
        
    train_errors.append(np.mean(run_train_errors))
    print("Polynomial order: " + str(poly_order) + " Average % train error: "+ str(np.mean(run_train_errors)))
    test_errors.append(np.mean(run_test_errors))
    test_sd.append(np.std(run_test_errors))
    train_sd.append(np.std(run_train_errors))

Polynomial order: 1 Average % train error: 3.543291207313794
Polynomial order: 2 Average % train error: 0.3085506856681903
Polynomial order: 3 Average % train error: 0.08133906964237698
Polynomial order: 4 Average % train error: 0.04907233127184728
Polynomial order: 5 Average % train error: 0.043694541543425655
Polynomial order: 6 Average % train error: 0.03562785695079323
Polynomial order: 7 Average % train error: 0.03764452809895134


In [51]:
# Create a df to store the error results
errors_df = pd.DataFrame()
errors_df['d'] = np.array(range(1,8))
errors_df['Train Error %'] = train_errors
errors_df['+/- Train %'] = train_sd
errors_df['Test Error %'] = test_errors
errors_df['+/- Test %'] = test_sd

In [52]:
errors_df

,d,Train Error %,+/- Train %,Test Error %,+/- Test %
0,1,3.543291,0.177674,6.451613,0.737559
1,2,0.308551,0.078218,3.706989,0.484013
2,3,0.081339,0.044076,3.336022,0.388243
3,4,0.049072,0.018666,3.282258,0.492889
4,5,0.043695,0.021204,3.336022,0.418691
5,6,0.035628,0.025957,3.413978,0.398539
6,7,0.037645,0.025367,3.440860,0.327472


In [54]:
# Export to csv
errors_df.to_csv('one-v-one-errors_df.csv',index=False)

### 1.2 Cross-Validation

In [59]:
d_stars = []
test_errors = []

for run in range(20):

    # Split the data into 80% training, 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
    
    # Initialise
    best_error = 500
    best_d = 0

    for d_ in range(1,8):
       
        error = 0 
        
        # Implement cross-validation
        kfold = KFold(5, True, 1)

        for train_index, test_index in kfold.split(X_train):
            Xtrain, Xtest = X_train[train_index], X_train[test_index]
            ytrain, ytest = y_train[train_index], y_train[test_index]
            clf = KernelPerceptron(Xtrain, ytrain, Xtest, ytest, kernel_type='p', d=d_, epochs=8)
            train_error = clf.train()
            error += clf.test()
        
        if error/5 < best_error:
            best_error = error/5
            best_d = d_
            
    # Once all the polynomial orders considered, retrain on full 80% using d*
    clf = KernelPerceptron(X_train, y_train, X_test, y_test, kernel_type='p', d=best_d, epochs=5)
    train_error = clf.train()
    test_error = clf.test()
    print("Run: " + str(run) + " Test Error: " + str(test_error) + " d*:  " + str(best_d))
    test_errors.append(test_error)
    d_stars.append(best_d)

Run: 0 Test Error: 2.956989247311828 d*:  4
Run: 1 Test Error: 4.408602150537634 d*:  4
Run: 2 Test Error: 5.43010752688172 d*:  3
Run: 3 Test Error: 3.118279569892473 d*:  4
Run: 4 Test Error: 3.4408602150537635 d*:  4
Run: 5 Test Error: 3.2795698924731185 d*:  5
Run: 6 Test Error: 4.354838709677419 d*:  3
Run: 7 Test Error: 3.870967741935484 d*:  6
Run: 8 Test Error: 3.2795698924731185 d*:  3
Run: 9 Test Error: 2.849462365591398 d*:  3
Run: 10 Test Error: 2.903225806451613 d*:  4
Run: 11 Test Error: 3.225806451612903 d*:  5
Run: 12 Test Error: 3.118279569892473 d*:  5
Run: 13 Test Error: 3.225806451612903 d*:  5
Run: 14 Test Error: 3.978494623655914 d*:  3
Run: 15 Test Error: 3.4408602150537635 d*:  6
Run: 16 Test Error: 2.903225806451613 d*:  6
Run: 17 Test Error: 2.4731182795698925 d*:  3
Run: 18 Test Error: 3.387096774193549 d*:  3
Run: 19 Test Error: 3.225806451612903 d*:  4


In [60]:
print("Mean test error: " + str(np.mean(test_errors)) + " +/- " + str(np.std(test_errors)))
print("Mean d*: " + str(np.mean(d_stars)) + " +/- " + str(np.std(d_stars)))

Mean test error: 3.4435483870967745 +/- 0.659665588662744
Mean d*: 4.15 +/- 1.061838029079765
